In [1]:
print("Let's Go!" )
from PIL import Image
import torch
from torchvision import transforms
import os
import shutil
import numpy as np
from torch.optim import Adam
from torch.nn import ReLU
from torch.nn import LayerNorm
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from torch.nn import BatchNorm2d
from torch.nn import Parameter
from torch.nn import Linear
from torch.nn import DataParallel
from torch.nn import Sequential
from torch.nn import ConvTranspose2d
from torch.nn import GRUCell
from torch.nn import Module
from torch.nn import Flatten
from torch.nn.init import normal_
from torch.utils.data import Dataset, DataLoader
from torch.nn.functional import softmax
from torch.nn import Conv2d
from torch.nn import Conv1d
from torch.nn import MaxPool2d
from torch.nn import MSELoss
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import CosineAnnealingLR
import sys
from tqdm import tqdm
from collections import OrderedDict
from torchvision.transforms import InterpolationMode
from sklearn.metrics import adjusted_rand_score
torch.manual_seed(0)
np.random.seed(0)
print("torch version:" , torch.__version__)

Let's Go!
torch version: 2.1.2


In [2]:
images_train = '/kaggle/input/clevertex6-train-images/CLEVERTex6_train_images'
CROP_SIZE  = (192 , 192)
ENCODER_RESOLUTION = (128 , 128)
DECODER_RESOLUTION = (8 , 8)

class ImageDatasetSlotAttention(Dataset):
    def __init__(self, images_folder , transform):
        super(ImageDatasetSlotAttention , self).__init__()
        self.images_folder = images_folder
        self.folder = images_folder
        self.transform = transform
        self.all_data =  self.build_data(images_folder)
        print("data size:" , self.all_data.shape)


    def build_data(self , folder_path):
        files = os.listdir(folder_path)
        files = sorted(files)
        data = []
        print("Loading data!")
        for file in tqdm(files):
            image = Image.open(f'{self.folder}/{file}').convert('RGB')
            data.append(self.transform(image))
        print("Loaded data!")
        return np.array(data)


    def __len__(self):
        return len(os.listdir(self.images_folder))
    def __getitem__(self , idx):
        return self.all_data[idx]

transform = transforms.Compose([
  transforms.PILToTensor(),
  transforms.ConvertImageDtype(torch.float),
  transforms.Normalize((0.5 ,  0.5 , 0.5), (0.5, 0.5 , 0.5)),
])

train_data = ImageDatasetSlotAttention(images_folder = images_train , transform = transform)
print("tensor max-min" , train_data[0].max().item() , train_data[0].min().item())

Loading data!


100%|██████████| 21361/21361 [01:49<00:00, 195.30it/s]


Loaded data!
data size: (21361, 3, 128, 128)
tensor max-min 0.9686274528503418 -0.8666666746139526


In [3]:
# At training time, we use T = 3 iterations of Slot Attention.
# number of slots K: we use K = 7 slots for
# N for us is 128 and D_inputs = 64 which is the final number of feature maps






class Block(Module):
    def __init__(self, channels):
        super(Block, self).__init__()
        self.downsample = Sequential(OrderedDict([
            ('conv1' , Conv2d(channels, channels, kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn1'   , BatchNorm2d(channels)),
            ('relu1' , ReLU()),
            ('conv2' , Conv2d(channels, channels, kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn2'   , BatchNorm2d(channels))
        ]))
        
        self.relu = ReLU()

    def forward(self, x):
        return self.relu(x + self.downsample(x))

class BlockUp(Module):
    def __init__(self , in_channels , out_channels):
        super(BlockUp, self).__init__()
        
        self.downsample = Sequential(OrderedDict([
            ('conv1' , Conv2d(in_channels , out_channels , kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn1'   , BatchNorm2d(out_channels)),
            ('relu1' , ReLU()),
            ('conv2' , Conv2d(out_channels , out_channels , kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn2'   , BatchNorm2d(out_channels)),
        ]))
        self.skip = Sequential(OrderedDict([
            ('conv1' , Conv2d(in_channels , out_channels, kernel_size = 5 , stride = 1 , padding = 2)),
            ('bn1'   , BatchNorm2d(out_channels))
        ]))
        self.relu = ReLU()



    def forward(self, x):
        return self.relu(self.skip(x) + self.downsample(x))
    
    
class ResNet18(Module):
    def __init__(self):

        super(ResNet18, self).__init__()
#         layer1 = self.__make_layer(64, 64)
#         layer2 = self.__make_layer(64, 64)
#         layer3 = self.__make_layer(64, 64)
#         layer4 = self.__make_layer(64, 64)
    
        layer0 = Sequential(OrderedDict([
         ('conv' , Conv2d(3 , 16 , kernel_size = 5 , stride = 1 , padding = 2)),
         ('bn'   , BatchNorm2d(16)),
         ('relu' , ReLU())
        ]))

        self.resnet = Sequential(OrderedDict([
            ('layer0' , layer0),
            ('block1' , Block(16)),
            ('block2' , Block(16)),
            ('block3' , BlockUp(16 , 32)),
            ('block4' , Block(32)),
            ('block5' , Block(32)),
            ('block6' , BlockUp(32 , 64)),
            ('block7' , Block(64)),
            ('block8' , Block(64))
        ]))


    def forward(self, x):
        return self.resnet(x)


In [4]:


class ObjectDiscovery(Module):
    def __init__(self , encoder_resolution ,  decoder_resolution , T   , K , D_slots):
        super(ObjectDiscovery , self).__init__()
        print(f'Initialized ObjectDiscovery!')
        self.layers = Sequential(OrderedDict([
          ("encoder" , ImageEncoder(resolution = encoder_resolution , T = T,
                                    K = K , D_slots = D_slots )),
          ("decoder" , SlotAttentionDecoder(resolution = decoder_resolution , K = K , D_slots = D_slots))
        ]))
    def forward(self , image):
        return self.layers(image)

class PositionEncoder(Module):
    def __init__(self, output_dim , resolution):
        super(PositionEncoder , self).__init__()
        self.linear =  Linear(in_features = 4 , out_features = output_dim)
        # above is equivalent to a linear layer
        self.grid = Parameter(data = PositionEncoder.build_grid(resolution) , requires_grad = False)
        print("Grid shape:" , self.grid.shape)

    @staticmethod
    def build_grid(resolution):
        ranges = [np.linspace(start = 0.0 , stop = 1.0 , num = dimension) for dimension in resolution] # dim = (2 , 128)
        grid = np.meshgrid(*ranges , sparse = False, indexing = "ij") # dim = (128 , 128)
        # row[i] of grid[0] has all elements i / 127 and col[j] of grid[1] has all elements j / 127
        grid = np.stack(grid , axis = -1) # dim = (64 , 64 , 2) to match conv later
        grid = np.expand_dims(grid, axis = 0) # dim = (1 , 64 , 64 , 2) for batch dimension
        grid = grid.astype(np.float32) # PyTorch throws an error later otherwise
        return torch.tensor(np.concatenate([grid , 1.0 - grid] , axis = 3)) # (1 , 64 , 64 , 4)


    def forward(self, x): # x has shape (batch , 64 , 64 , D_inputs)
        return x + self.linear(self.grid)


class ImageEncoder(Module):
    def __init__(self , resolution , T  ,  K , D_slots):
        super(ImageEncoder , self).__init__()
        print(f'Initialized ImageEncoder! resolution: {resolution}')
        D_inputs = 64
        self.encoder_cnn = ResNet18()
        down_resolution = (128 , 128)
        positional_encoder = PositionEncoder(output_dim =  D_inputs , resolution = down_resolution)


        slot_attention = SlotAttention(T = T , K = K , D_slots = D_slots)
        self.pos_encode_feedforward_slotattn = Sequential(OrderedDict([
            ("pos_encoder" , positional_encoder), #  (batch , 64 , 64 , D_inputs)
            ("flatten" , Flatten(start_dim = 1 , end_dim = 2)), # (batch , 64 * 64 , D_inputs)
            ("layer_norm" ,  LayerNorm(normalized_shape = 64)), # (batch , 64 * 64 , D_inputs)
            ("linear1:" , Linear(in_features = 64 , out_features = 128)),
            ("relu1:"   , ReLU()),
            ("linear2:" , Linear(in_features = 128 , out_features = 128)),
            ("slot_attention" , slot_attention) #  (batch , K , D_slots)
        ]))
    # feature map has shape (channels , h , w)
    # N is h * w i.e. each pixel is a different feature "vector", size of this vector of D_inputs = # of channels
    # channels = D_inputs , h * w = N
    def forward(self , x): # x is aimge with shape (batch , 3 , 126 , 128)
        x = self.encoder_cnn(x).permute(dims = (0 , 2 , 3 , 1))  # (batch , 64 , 64 , D_inputs)
        return self.pos_encode_feedforward_slotattn(x)



class SlotAttention(Module):
    def __init__(self , T, K , D_slots  , epsilon = 1e-8):
        super(SlotAttention , self).__init__()
        print(f'Initialzing SlotAttention parameters: T: {T} , K: {K} , D_slots: {D_slots}')
        self.T = T
        self.K = K
        self.D_slots = D_slots
        self.epsilon = epsilon

        self.norm_inputs = LayerNorm(normalized_shape =  128)
        self.query_from_slots  = Sequential(OrderedDict([
            ("NormSlots" , LayerNorm(normalized_shape = D_slots)),
            ("SlotQuery" , Linear(in_features = D_slots  , out_features = D_slots , bias = False))
        ]))


#         self.slots_init = Parameter(data = normal_(torch.empty(1 , K , D_slots)) , requires_grad = True)

        self.init_latents = Parameter(normal_(torch.empty(1 , self.K , self.D_slots)))



        self.keys_from_inputs  = Linear(in_features = 128 , out_features = D_slots , bias = False)
        self.vals_from_inputs  = Linear(in_features = 128 , out_features = D_slots , bias = False)
        self.gru = GRUCell(input_size = D_slots , hidden_size = D_slots)
        self.norm_feed = Sequential(OrderedDict([
            ("layer_norm" ,  LayerNorm(normalized_shape = D_slots)),
            ("linear1" , Linear(in_features = D_slots , out_features = 256)),
            ("relu1"   , ReLU()),
            ("linear2" , Linear(in_features = 256 , out_features = D_slots))
        ]))


    def forward(self, inputs):
        batch_size = inputs.shape[0]
        inputs = self.norm_inputs(inputs)
        keys = self.keys_from_inputs(inputs)
        vals = self.vals_from_inputs(inputs)
        slots = self.init_latents.repeat(batch_size , 1 , 1)
        for _ in range(self.T):
            slots_prev = slots
            queries = self.query_from_slots(slots)
            attn_logits = (self.D_slots ** -0.5) * torch.einsum('bnc,bmc->bnm', keys , queries)
            attn = F.softmax(attn_logits , dim = -1)
            attn = attn + self.epsilon
            attn = attn / torch.sum(attn, dim = 1, keepdim = True)
            updates = torch.einsum('bnm,bnc->bmc' , attn , vals)
            slots = self.gru(
                updates.view(batch_size * self.K , self.D_slots),
                slots_prev.view(batch_size * self.K , self.D_slots),
            )
            slots = slots.view(batch_size , self.K , self.D_slots)
            slots = slots + self.norm_feed(slots)

        return slots


class SlotAttentionDecoder(Module):
    def __init__(self , resolution , K , D_slots):
        super(SlotAttentionDecoder , self).__init__()
        print(f'Initialized SlotAttentionDecoder! resolution: {resolution}')
        print(f'Decoder parameters: K: {K} , D_slots: {D_slots}')
        self.resolution = resolution
        self.positional_encoder = PositionEncoder(output_dim = D_slots , resolution = resolution)

        self.K = K
        self.D_slots = D_slots

        self.decoder_cnn = Sequential(OrderedDict([
            ("conv1" , ConvTranspose2d(in_channels = D_slots , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu1' , ReLU()),
            ("conv2" , ConvTranspose2d(in_channels = 64 , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu2' , ReLU()),
            ("conv3" , ConvTranspose2d(in_channels = 64 , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu3' , ReLU()),
            ("conv4" , ConvTranspose2d(in_channels = 64 , out_channels = 64 , kernel_size = 5 , stride = 2 , padding = 2 , output_padding = 1)),
            ('relu4' , ReLU()),
            ("conv5" , ConvTranspose2d(in_channels = 64 , out_channels = 4 , kernel_size = 1 , stride = 1)),
        ]))


    def forward(self , slots):
        # slots comes form the ImageEncoder shape : (batch , K , D_slots)
        batch_size = slots.shape[0]
        x =  slots.view(batch_size * self.K , 1 , 1 , self.D_slots)
        x = x.repeat(1 , self.resolution[0] , self.resolution[1] , 1)
        x = self.positional_encoder(x).permute(0 , 3 , 1 , 2)
        x = self.decoder_cnn(x) # dim = (batch * K , 4 , 128 , 128)
        x = x.view(batch_size, self.K , 4 , 128 , 128)# dim = (batch , K , 4 , 128 , 128)
        recons , masks = x[: , : , : 3 , : , :]  , x[: , : , -1 : , : , :]
        masks = F.softmax(masks , dim = 1) # dim = (batch , K , 1 , 128, 128)
        reconstructed = torch.sum(recons * masks , dim = 1) # dim = (batch , 3 , 128, 128)
        return reconstructed

In [5]:
BATCH_SIZE = 64
train_loader = DataLoader(train_data , batch_size = BATCH_SIZE , shuffle = True , num_workers = 1 , pin_memory = True)
print("image shape:" , train_data[0].shape)
print("train data size  :" , len(train_data))
print("# train batches  :" , len(train_loader))
print("mini-batch size  :" , BATCH_SIZE)
print("num_workers train:" , train_loader.num_workers)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:" , device)

image shape: (3, 128, 128)
train data size  : 21361
# train batches  : 334
mini-batch size  : 64
num_workers train: 1
Using device: cuda


In [6]:
def create_directory(directory_path):
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)
        print(f"Directory '{directory_path}' deleted.")

    os.makedirs(directory_path)
    print(f"Directory '{directory_path}' created successfully.")

def save_model(epoch , model , optimizer , train_loss , path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        }, path)


def show_gpus():
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print("Number of available GPUs:", num_gpus)


        for i in range(num_gpus):
            gpu_properties = torch.cuda.get_device_properties(i)
            print(f"GPU {i}: {gpu_properties.name}, Memory: {gpu_properties.total_memory / 1024**3:.2f}GB")
    else:
        print("CUDA is not available. Running on CPU.")

def get_loss(model_ , data_loader):
    model_.eval()
    loss = 0.0
    with torch.no_grad():
        for images in data_loader:
            images = images.to(device)
            reconst = model_(images)
            loss += criterion(reconst , images).item()
    return loss / len(data_loader)



In [7]:


torch.autograd.set_detect_anomaly(True)
directory = 'results'

def train_loop(results_folder , EPOCH , lr):
    show_gpus()
    model = DataParallel(ObjectDiscovery(encoder_resolution = (128 , 128) , decoder_resolution = (8 , 8),
                    T = 3  , K = 7 , D_slots = 128))
    model.to(device)
    optimizer = Adam(model.parameters() , lr = lr)
    criterion = MSELoss()
    scheduler = CosineAnnealingLR(optimizer , T_max = 160)
    for epoch in range(0 , EPOCH):
        model.train()
        train_loss = 0
        print(f"epoch     : {epoch + 1}")
        for images  in train_loader:
            images = images.to(device)
            reconst = model(images)
            optimizer.zero_grad()
            loss = criterion(reconst , images)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        scheduler.step()

        print(f"train loss: {train_loss}")
        print(f'learning rate:' , optimizer.param_groups[0]['lr'])
        print("__________________________________________________________________________________________")
        save_model(epoch , model , optimizer , train_loss  , path = f'{directory}/model_{epoch + 1}')

torch.cuda.empty_cache()
create_directory(directory)
train_loop(results_folder = directory , EPOCH = 80 , lr = 0.001)
print("Successfully trained?")

Directory 'results' created successfully.
Number of available GPUs: 2
GPU 0: Tesla T4, Memory: 14.75GB
GPU 1: Tesla T4, Memory: 14.75GB
Initialized ObjectDiscovery!
Initialized ImageEncoder! resolution: (128, 128)
Grid shape: torch.Size([1, 128, 128, 4])
Initialzing SlotAttention parameters: T: 3 , K: 7 , D_slots: 128
Initialized SlotAttentionDecoder! resolution: (8, 8)
Decoder parameters: K: 7 , D_slots: 128
Grid shape: torch.Size([1, 8, 8, 4])
epoch     : 1
train loss: 0.12863549594572204
learning rate: 0.0009999036202410325
__________________________________________________________________________________________
epoch     : 2
train loss: 0.09812912012556356
learning rate: 0.0009996145181203615
__________________________________________________________________________________________
epoch     : 3
train loss: 0.09245876662299304
learning rate: 0.000999132805092358
__________________________________________________________________________________________
epoch     : 4
train loss: 0.0